In [1]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import functions as F

## 1.) Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language

In [2]:
languages = pd.DataFrame({'languages' : ['Python', 'Java', 'SQL', 'C++', 'Ruby', 'Haskell', 'JavaScript', 'C#', 'Scala']})


spark = pyspark.sql.SparkSession.builder.getOrCreate()
df = spark.createDataFrame(languages)
df.show()

+----------+
| languages|
+----------+
|    Python|
|      Java|
|       SQL|
|       C++|
|      Ruby|
|   Haskell|
|JavaScript|
|        C#|
|     Scala|
+----------+



- View the schema of the dataframe

In [3]:
df.printSchema()

root
 |-- languages: string (nullable = true)



- Output the shape of the dataframe

In [4]:
print((df.count(), len(df.columns)))

(9, 1)


- Show the first 5 records in the dataframe

In [5]:
df.show(5)

+---------+
|languages|
+---------+
|   Python|
|     Java|
|      SQL|
|      C++|
|     Ruby|
+---------+
only showing top 5 rows



## 2.) Load the mpg dataset as a spark dataframe

- Create 1 column of output that contains a message like the one below:

    <mark>The 1999 audi a4 has a 4 cylinder engine.</mark>

    For each vehicle.

In [6]:
from pydataset import data
mpg = data('mpg')
df = spark.createDataFrame(mpg)
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [7]:
df.select(F.concat(F.lit('The '), df.year, 
                F.lit(' '),df.manufacturer,
                F.lit(' ') ,df.model,F.lit(' has a '), df.cyl,
                F.lit('cylinder engine.'))
                .alias('sentence')).show(truncate=False)

+-------------------------------------------------------------+
|sentence                                                     |
+-------------------------------------------------------------+
|The 1999 audi a4 has a 4cylinder engine.                     |
|The 1999 audi a4 has a 4cylinder engine.                     |
|The 2008 audi a4 has a 4cylinder engine.                     |
|The 2008 audi a4 has a 4cylinder engine.                     |
|The 1999 audi a4 has a 6cylinder engine.                     |
|The 1999 audi a4 has a 6cylinder engine.                     |
|The 2008 audi a4 has a 6cylinder engine.                     |
|The 1999 audi a4 quattro has a 4cylinder engine.             |
|The 1999 audi a4 quattro has a 4cylinder engine.             |
|The 2008 audi a4 quattro has a 4cylinder engine.             |
|The 2008 audi a4 quattro has a 4cylinder engine.             |
|The 1999 audi a4 quattro has a 6cylinder engine.             |
|The 1999 audi a4 quattro has a 6cylinde

- Transform the trans column so that it only contains either manual or auto.

In [8]:
df.select(F.regexp_extract('trans', r'^([a-z]+)', 1).alias('trans_new')).show()

+---------+
|trans_new|
+---------+
|     auto|
|   manual|
|   manual|
|     auto|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|     auto|
|   manual|
|     auto|
|   manual|
|     auto|
|     auto|
|     auto|
|     auto|
|     auto|
+---------+
only showing top 20 rows



## 3.) Load the tips database

In [9]:
df = data('tips')
df = spark.createDataFrame(df)
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



- What percentage of observations are smokers?

In [10]:
(df.filter(df.smoker == 'Yes').count()) / df.count()

0.38114754098360654

- Create a column that contains the tip percentage

In [11]:
df.select('*',(df.tip/df.total_bill).alias('tip_percentage')).show()

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|0.18623962040332148|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|0.22805017103762829|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|0.11607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|0.13031914893617022|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2| 0.2185385656292287|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 0.1665043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|0

- Calculate the average tip percentage for each combination of sex and smoker.

In [12]:
df.groupby('sex','smoker').mean('tip').show()

+------+------+------------------+
|   sex|smoker|          avg(tip)|
+------+------+------------------+
|  Male|    No|3.1134020618556697|
|  Male|   Yes|3.0511666666666666|
|Female|    No| 2.773518518518518|
|Female|   Yes|2.9315151515151516|
+------+------+------------------+



## 4.) Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [16]:
from vega_datasets import data
weather = data.seattle_weather()
df = spark.createDataFrame(weather)
df.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



- Convert the temperatures to farenheight.

In [20]:
df.withColumn('temp_max', df.temp_max * 9 / 5 + 32).withColumn('temp_min', df.temp_min * 9 / 5 + 32).show()

+-------------------+-------------+------------------+------------------+----+-------+
|               date|precipitation|          temp_max|          temp_min|wind|weather|
+-------------------+-------------+------------------+------------------+----+-------+
|2012-01-01 00:00:00|          0.0|             55.04|              41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|             51.08|             37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|             53.06|             44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|             53.96|             42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|             48.02|             37.04| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|             39.92|             35.96| 2.2|   rain|
|2012-01-07 00:00:00|          0.0|             44.96|             37.04| 2.3|   rain|
|2012-01-08 00:00:00|          0.0|              50.0|             37.04| 2.0|    sun|
|2012-01-09 00:00:00|          4.3|        

- Which month has the most rain, on average?

In [21]:
df.withColumn('month', month('date')).groupBy('month').agg(mean('precipitation').alias('avg_monthly_rain')).sort('month').show()

NameError: name 'month' is not defined

- Which year was the windiest?

- What is the most frequent type of weather in January?

- What is the average high and low tempurature on sunny days in July in 2013 and 2014?

- What percentage of days were rainy in q3 of 2015?

- For each year, find what percentage of days it rained (had non-zero precipitation).